## So you want to do research on Twitter

This tutorial covers the basics of using the _twitter_ library in Python to perform searches using the REST API.  

__NB:__ There are a lot of different ways of accessing and processing Twitter data. I'm going to go through my workflow, which is a kludging together of methods assembled over the last four years.  There are probably better approaches to at least some parts of what I'm going through here, and if you know of one or find one, let me know!

## What's Twitter like?

Not to sound like a tech evangelist, but one neat thing about Twitter is that everyone uses and experiences it differently.  (Let's have an aside about everyone's familiarity with Twitter and what they use it for.)

Now, here's the interesting thing to me about using Twiter for research: Twitter is much more mundane than you think. It's a little paradoxical; most tweets on Twitter are nothing like most tweets that we see, because the tweets that pull big numbers (views, favs, RTs) are from bigger accounts and/or disseminated through the network broadly. But most tweets, like most of us, die in obscurity.

Here's an example of this.  Let's search for something really pragmatically empty, so we're going to get a scattering of topics. Specifically, let's look at tweets containing the word *the*:

https://twitter.com/search?l=&q=the&src=typd

The usual, common suspects show up in the default web-based search. Trump, 5 Seconds of Summer, news organizations, etc.  But that's because the Twitter algorithm defaults to show you a mix of recent and "top" tweets.  If you click on the little "Latest" tab, it gives you all the recent tweets for *the*, and it looks a lot different. 

https://twitter.com/search?f=tweets&vertical=default&q=the&src=typd

Here are the normies, the folks like you or me, or at least me, who chat with a few friends or try in vain to get noticed. Now, depending on your goals in your research, you might want the popular accounts and the viral tweets.  But if you're interested in how people talk, or how they interact, it's these normies who are your best friends.

PS: if you want to test out your searches before API-ing them, here's the advanced search link: https://twitter.com/search-advanced


## Initialization basics

We'll start by setting up a Twitter search object, and doing a basic search.

In [ ]:
import sys, os
from pprint import pprint                           #Important for reading through JSONs
from time import localtime,strftime,sleep,time      #Important for dealing with Twitter rate limits
import twitter

### Authorizing your account

The first thing you need is to perform an OAuth dance to tell Twitter who you are, and that you're cool. This is a slightly complicated step, involving identification both of a user conducting the searches and a client through which they're conducting them.

Hopefully you have already set up both your Twitter account and an app for accessing this data. If not, do it now; it's a pretty trivial process once you have a Twitter account, and can be set up in a few minutes here: https://apps.twitter.com/app/new

[Optional aside with link to set up app.]

Now that you have an app set up, we're going to input its access keys. Go to https://apps.twitter.com, click on your app, and then click on the "Keys and Access Tokens" tab.  Copy them in when prompted.

In [ ]:
cons_oauth_file = 'c.xxx'
if os.path.exists(cons_oauth_file):
    constoken, conssecret = twitter.read_token_file(cons_oauth_file)
else:
    constoken = raw_input("What is your app's 'Consumer Key'?").strip()
    conssecret = raw_input("What is your app's 'Consumer Secret'?").strip()
    wf = open(cons_oauth_file,'w'); wf.write(constoken+'\n'+conssecret); wf.close()

Now we'll authorize this app through your Twitter account. This code block will open a new tab, prompt to activate the app, and provide you with a PIN to enter back here. This OAuth registration is handled by the *twitter.oauth_dance* function.

In [ ]:
app_oauth_file = 'a.xxx'
if not os.path.exists(app_oauth_file):									#if user not authorized already
	twitter.oauth_dance("your app",constoken,conssecret,app_oauth_file)		#perform OAuth Dance
apptoken, appsecret = twitter.read_token_file(app_oauth_file)					#import user credentials

### Initializing a search object

Here's the core of the code, really -- we're setting up an object that uses your credentials to access the Twitter REST API and return structured Twitter data.  Thanks to the _twitter_ library, it's a single command, and then we can work with the resulting object, _tsearch_.

In [ ]:
tsearch = twitter.Twitter(auth=twitter.OAuth(apptoken,appsecret,constoken,conssecret))	#create search command

## Basics of the search object & API

Now that _tsearch_ is initialized, let's get to searching! Except, actually, let's hold up a second and talk about how to use the API.

Twitter provides fairly extensive (but often confusing) documentation for its REST API, available here: https://dev.twitter.com/rest/reference  Basically, the REST API handles searches of recent tweets (the last week, generally), as well as functions related to the social network structure, trending topics, and user information.

### An aside: what won't we be covering?

There is another Twitter API, the streaming API, which maintains a connection to Twitter and collects tweets as they occur.  I don't have enough experience with the streaming API to teach it, and many of its potential applications can be handled through the REST API.  I also won't be dealing with the POST features of the REST API, because we're focused on collecting existing data rather than generating our own.

### Rate limits

Perhaps the most important thing to know about for the REST API is that it is rate-limited. For each app/account combo, you can only perform so many searches per time period. I believe Twitter resets its counter every 15 minutes for all API calls; it definitely does for all the common ones.  If you try to search once you've hit the rate limit, the search will return an error. You will want to make sure your code doesn't persist much after it hits a rate limit, or else Twitter may get mad at your account (this is why I suggest setting up an account specifically for these searches, just in case).

There is also a return limit, both as a limit to the number of results any single search will give you, and a limit on how far back Twitter will search for you.  In general, you can't ask for more than 100 hits per individual search, though you can search reatedly to get more and more 100-hit blocks.  But after a while, Twitter says enough of that and stops giving you anything new.  We'll get to illustrations in a little bit, but for now, here are some commonly encountered limits, and the full list is available [here](https://dev.twitter.com/rest/public/rate-limits).

| Function call | limit per 15 minutes | max returns per search | max history | 
|---|---|---|
| GET search/tweets | 180 | 100 | 3000 / 7 days |
| GET statuses/user_timeline | 900 | ? | 3200 / infinite past |
| GET users/lookup | 900 | 100 | NA |

### Checking rate limits

Since we don't want to anger Twitter, let's talk about checking rate limits.  Your _tsearch_ object can check rate limits, and it's a nice way of getting used to the nested list format of the search results.

By the way, checking rate limits **is itself rate limited** (180 / 15 minutes)! 

In [ ]:
r = tsearch.application.rate_limit_status()  #Obtain rate limit status for all searches
pprint(r)

*application.rate_limit_status()* returns a set of nested dictionaries; one for each type of API request, sorted by request type. For each request, it lists the rate limit (*'limit'*), the number of API calls you have left before you hit that limit (*'remaining'*), and the Unix time at which your rate limit resets (*'reset'*).

Each API call type is tracked completely separately, and the rate-limit window is **15 minutes from your first call *specific to that request*.**  For instance, let's try doing a standard term-based search (GET search/tweets) and then check the rate limits again.

In [ ]:
res = tsearch.search.tweets(q='test',              #A test query
                            count=10,              #just want 10 hits back
                            result_type="recent")  #include all recent tweets, not only popular ones

r = tsearch.application.rate_limit_status()  #Obtain rate limit status for all searches
pprint(r['resources']['application'])        #rate limit query's rate limit
pprint(r['resources']['search'])             #search query's rate limit

We've now done two rate limit queries, so the remaining available in this window is 178. But we've only done one term-based query, so there are still 179 left on that.  But importantly, the two have different reset times, because the rate-limit reset was triggered by the first search from the previous code block, while the term search was triggered more recently.

By the way, the reset time is always given in Unix time, UTC (i.e., number of milliseconds since 1/1/1970 UTC). Converting to local human-readable time isn't too bad, using *localtime* and *strftime* from the Python *time* library:

In [ ]:
print('Rate limit reset:')
print(localtime(r['resources']['application']['/application/rate_limit_status']['reset']))
print(strftime('%I:%M:%S',localtime(r['resources']['application']['/application/rate_limit_status']['reset'])))
print('\n')
print('Term search reset:')
print(strftime('%I:%M:%S',localtime(r['resources']['search']['/search/tweets']['reset'])))

### Dealing with the rate limit

In this tutorial, we won't push up against the rate limit much, but for many tasks, you're going to have to deal intelligently with it. Let me share a few quick tricks, and then we'll move on to real searches.

**Trick 1: *time.sleep()* **

The most basic solution is to take advantage of the *sleep* function, which tells Python to hold on for some specified length of time. When your 'remianing' value drops to (or near) zero, record the 'reset' time, and subtract the current time from it (*time.time()*).  I generally add a little buffer time, too, in case my system clock is off from Twitter's.

One note: if you haven't triggered a rate limit window by doing a search, the reset value will be 15 minutes from now, so build in a check to make sure that you're not sitting around infinitely.  Any suggestions for how to best handle that?

In [ ]:
print 'Reset time:  ',r['resources']['application']['/application/rate_limit_status']['reset']
print 'Current time:',time()
print 'Wait time:   ',r['resources']['application']['/application/rate_limit_status']['reset'] - time() + 30

**Trick 2: interleave requests**

Because the rate limits are triggered separately for each search type, if you're doing multiple search types (say, term and user searches), you can switch back and forth, working with one until the limit's hit, then going to the other.

In [ ]:
temp = tsearch.followers.list()
r = tsearch.application.rate_limit_status()
pprint(r['resources']['followers'])

**Trick 3: process in the moment**

If you're doing any substantial tweet processing within Python, you can take advantage of the rate limit to do time-consuming processing while you wait. Since the rate limit window is triggered by the first search, you want to make sure to put the first search as early in your code as possible, but subsequent searches can come either before or after processing wihtout consequnces.

**Pitfall 1: track limits internally, confirm occasionally**

One small thing: since the rate limit query is itself rate-limited, you want to ration your calls to it, especially if you're performing more than 180 searches by interleaving request types. I keep an internal count of the number of times I called each search, and then every 3 or so searches, check back with *application.rate_limit_status()* to make sure twitter and I are in agreement.

### What happens if we pass the rate limit?

All right, let's give in to our darker motivations and wantonly exceed the rate limit to find out the consequences.



In [ ]:
r = tsearch.application.rate_limit_status()
for i in range(0,r['resources']['followers']['/followers/list']['remaining']):
    temp = tsearch.followers.list()

In [ ]:
try:
    temp = tsearch.followers.list()
except:
    print "error"

Ah, it just kinda hangs there! So maybe that sets up:

**Trick 5: always *try* **

Unfortunately, the *twitter* library doesn't handle errors very well, in part, I think, because Twitter changes how the API handles errors from time to time.  So, if you're going to be doing a large number of searches and/or automating your system, be sure to build some safety valves in the form of *try-catch-except* statements around all the non-manual searches.

Note that sometimes the system itself craps out, so not all errors are due to rate limit violations. For important searches that I'm not observing, after a failed search, I have it wait a few seconds and try again (and maybe even another wait-try cycle), before doing a 15-minute wait cycle to make sure all the rate limits have refreshed.  You can probably work out a better system than that.

## Actually searching

Okay, enough of all the boring limitations. Let's take this puppy out on the road in Section 2!